In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('Train.csv',encoding='utf-8')

In [5]:
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
df['length']=df['tweet'].apply(len)

In [2]:
import string
from nltk.corpus import stopwords
import re

In [20]:
def clean_text(msg):
    msg = re.sub('@[\w]*','',msg)
#     msg=msg.translate(str.maketrans('','',string.punctuation))
#     msg=msg.translate(str.maketrans('','',string.digits))
    msg = re.sub('[^a-zA-Z ]','',msg)
    msg = " ".join([words for words in msg.split() if len(words)>1])
#     print(msg)
    return [word for word in msg.lower().split() if word not in stopwords.words('english')]

In [21]:
df['token']=df['tweet'].apply(clean_text)

In [2]:
df.to_csv('cleaned.csv')

NameError: name 'df' is not defined

In [4]:
df = pd.read_csv('cleaned.csv',encoding='utf-8')
df.head()

,Unnamed: 0,id,label,tweet,length,token
0,0,1,0,@user when a father is dysfunctional and is s...,102,"['father', 'dysfunctional', 'selfish', 'drags'..."
1,1,2,0,@user @user thanks for #lyft credit i can't us...,122,"['thanks', 'lyft', 'credit', 'cant', 'use', 'c..."
2,2,3,0,bihday your majesty,21,"['bihday', 'majesty']"
3,3,4,0,#model i love u take with u all the time in ...,86,"['model', 'love', 'take', 'time', 'ur']"
4,4,5,0,factsguide: society now #motivation,39,"['factsguide', 'society', 'motivation']"


In [5]:
from nltk.stem import WordNetLemmatizer

In [6]:
lemm = WordNetLemmatizer()

In [7]:
df['tweet_lemm']=df['token'].apply(lambda x: " ".join([lemm.lemmatize(word) for word in eval(x)]))

In [8]:
df.head()

,Unnamed: 0,id,label,tweet,length,token,tweet_lemm
0,0,1,0,@user when a father is dysfunctional and is s...,102,"['father', 'dysfunctional', 'selfish', 'drags'...",father dysfunctional selfish drag kid dysfunct...
1,1,2,0,@user @user thanks for #lyft credit i can't us...,122,"['thanks', 'lyft', 'credit', 'cant', 'use', 'c...",thanks lyft credit cant use cause dont offer w...
2,2,3,0,bihday your majesty,21,"['bihday', 'majesty']",bihday majesty
3,3,4,0,#model i love u take with u all the time in ...,86,"['model', 'love', 'take', 'time', 'ur']",model love take time ur
4,4,5,0,factsguide: society now #motivation,39,"['factsguide', 'society', 'motivation']",factsguide society motivation


In [7]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# cv = CountVectorizer()
# words = cv.fit_transform(df['tweet_lemm'])
# tfidf = TfidfTransformer()
# bow = tfidf.fit_transform(words)
# bow = bow.toarray()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df=0.9,min_df=5,ngram_range=(1,2))
bow = tfidf_vec.fit_transform(df['tweet_lemm'])
bow = bow.toarray()

In [10]:
y = np.array(df['label'])

In [11]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Flatten ,Dense,Dropout
from tensorflow.keras.models import Sequential

In [16]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=bow.shape[1:]))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

In [17]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(bow,y,validation_split=0.1,epochs=10)

Train on 28765 samples, validate on 3197 samples
Epoch 1/10
28765/28765 [==============================] - 18s 628us/sample - loss: 0.1805 - accuracy: 0.9415 - val_loss: 0.1244 - val_accuracy: 0.9606
Epoch 2/10
28765/28765 [==============================] - 15s 506us/sample - loss: 0.0884 - accuracy: 0.9674 - val_loss: 0.1269 - val_accuracy: 0.9618
Epoch 3/10
11840/28765 [===========>..................] - ETA: 7s - loss: 0.0593 - accuracy: 0.9783